In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pickle
file_path = './splitData.pickle'
with open(file_path, 'rb') as f:
    texts_train = pickle.load(f)
    texts_val = pickle.load(f)
    texts_test = pickle.load(f)
    labels_train = pickle.load(f)
    labels_val = pickle.load(f)
    labels_test = pickle.load(f)

all_texts  = texts_train  + texts_val  + texts_test
all_labels = labels_train + labels_val + labels_test
# check the ratio of each dataset
total = len(texts_train) + len(texts_val) + len(texts_test)
len(texts_train)/total, len(texts_val)/total, len(texts_test)/total

(0.5997757847533632, 0.20011210762331838, 0.20011210762331838)

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
import torch
from transformers import pipeline
pipe = pipeline('feature-extraction',model='roberta-base',truncation=True)# Truncation is needed to handle longer documents over 512 tokens


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Show classifier performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def showPerformance(labels_true, labels_predicted):
    print(f'accuracy ={accuracy_score(labels_true, labels_predicted):.3f}')
    print(f'precision={precision_score(labels_true, labels_predicted, average="macro"):.3f}')
    print(f'recall   ={recall_score(labels_true, labels_predicted, average="macro"):.3f}') 
    print(f'f1       ={f1_score(labels_true, labels_predicted, average="macro"):.3f}')
    return ;

# C

### Creating a Dataset

In [5]:
# correspond labels(songer names) with integers
label2id = {label: i for i, label in enumerate(set(labels_train))}
id2label = {id:label for label,id in label2id.items()}
labels_train_int = [label2id[l] for l in labels_train]
labels_val_int = [label2id[l] for l in labels_val]

In [6]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def createDataset(texts, labels_int):
    encoded_texts = []
    for t in texts:
        encoded_texts.append(tokenizer.encode(t,truncation=True))#,max_length=128))### max_length need to be changed
    
    dataset = Dataset.from_dict({
        "input_ids":encoded_texts,
        "labels": labels_int
    })
    return dataset

train_dataset = createDataset(texts_train, labels_train_int)
validation_dataset = createDataset(texts_val, labels_val_int)

In [7]:
from transformers import set_seed, AutoModelForSequenceClassification

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained('roberta-base',id2label=id2label)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [12]:
# 3 sets of parameters
# Batch size cannot be over 16 due to memory issue.

# make model sensitive to each data by decreasing batch_size and increasing learning_rate
param_a = {"learning_rate":1e-3,
           "epochs": 10,
           "batch_size":4}

# make model less sensitive to each data by increasing batch_size and decreasing learning_rate. increase epochs instead to help converge.
param_b = {"learning_rate":1e-5,
           "epochs":50 ,
           "batch_size":16}

# middle of A and B
param_c = {"learning_rate":1e-4,
           "epochs": 25,
           "batch_size":8}

# param_b gained the best result (f1 = 0.586), so minor change from param_b
param_d = {"learning_rate":1e-5,
           "epochs":50 ,
           "batch_size":8}

param_e = {"learning_rate":1e-5,
           "epochs":75 ,
           "batch_size":16}       

In [9]:
from transformers import TrainingArguments, Trainer,DataCollatorWithPadding

def createTrainer(p,name):
  training_args = TrainingArguments(
      output_dir                 = f"model_{name}", # HuggingFace wants a name for your model
      evaluation_strategy        = "epoch", # How often we want to evaluate the model
      learning_rate              = p["learning_rate"], # Hyperparameter
      per_device_train_batch_size= p["batch_size"], # Hyperparameter
      per_device_eval_batch_size = p["batch_size"], # Hyperparameter
      num_train_epochs           = p["epochs"] # Hyperparameter
      # weight_decay=weight_decay, # Hyperparameter
  )
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    # training
  trainer = Trainer(
      model=model, # The model you want to train
      args=training_args, # The various training arguments set up above
      train_dataset=train_dataset, # The data to use to update the weights
      eval_dataset=validation_dataset, # The data to use 
      tokenizer=tokenizer, # The tokenizer used on the data
      data_collator=data_collator, # A data collator that does clever things moving data around
      )
  return trainer


In [10]:
# Show classifier performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def showPerformance(labels_true, labels_predicted):
    print(f'accuracy ={accuracy_score(labels_true, labels_predicted):.3f}')
    print(f'precision={precision_score(labels_true, labels_predicted, average="macro"):.3f}')
    print(f'recall   ={recall_score(labels_true, labels_predicted, average="macro"):.3f}') 
    print(f'f1       ={f1_score(labels_true, labels_predicted, average="macro"):.3f}')
    return ;

def evalModel(trainer,name):
    # measuring metrics with validation set
  predictions, label_ids, metrics = trainer.predict(validation_dataset)
  labels_predicted = predictions.argmax(axis=1)
  print(f'----------model{name}---------')
  showPerformance(label_ids, labels_predicted)
  return ;


In [13]:
trainer = createTrainer(param_e,"e")
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 1070
  Num Epochs = 75
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5025
  Number of trainable parameters = 124653322


Epoch,Training Loss,Validation Loss
1,No log,2.113155
2,No log,1.731240
3,No log,1.522281
4,No log,1.476593
5,No log,1.389152
6,No log,1.325397
7,No log,1.559131
8,1.283300,1.469840
9,1.283300,1.464493
10,1.283300,1.511230


***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
Saving model checkpoint to model_e/checkpoint-500
Configuration saved in model_e/checkpoint-500/config.json
Model weights saved in model_e/checkpoint-500/pytorch_model.bin
tokenizer config file saved in model_e/checkpoint-500/tokenizer_config.json
Special tokens file saved in model_e/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples = 357
  Batch size = 16
***** Running Evaluation *****
  Num examples 

TrainOutput(global_step=5025, training_loss=0.15000837858618046, metrics={'train_runtime': 9457.6835, 'train_samples_per_second': 8.485, 'train_steps_per_second': 0.531, 'total_flos': 2.1116178832896e+16, 'train_loss': 0.15000837858618046, 'epoch': 75.0})

In [14]:
evalModel(trainer,"e")

***** Running Prediction *****
  Num examples = 357
  Batch size = 16


----------modele---------
accuracy =0.583
precision=0.611
recall   =0.590
f1       =0.581


In [17]:
trainer.save_model("./paramE")


Saving model checkpoint to ./paramE
Configuration saved in ./paramE/config.json
Model weights saved in ./paramE/pytorch_model.bin
tokenizer config file saved in ./paramE/tokenizer_config.json
Special tokens file saved in ./paramE/special_tokens_map.json


In [ ]:
!zip -r /content/modelE.zip /content/model_e
!zip -r /content/paramE.zip /paramE

In [ ]:
from google.colab import files
files.download("/content/modelE.zip")
files.download("/content/paramE.zip")

## creating test dataset

In [15]:
labels_test_int = [label2id[l] for l in labels_test]
test_dataset = createDataset(texts_test, labels_test_int)

In [20]:
t_predictions, t_label_ids, t_metrics = trainer.predict(test_dataset)
t_labels_predicted = t_predictions.argmax(axis=1)
print(f'----------test---------')
showPerformance(t_label_ids, t_labels_predicted)

with open("test_resultsE.pickle","wb") as f:
  pickle.dump(t_labels_predicted, f)
  pickle.dump(t_label_ids, f)
from google.colab import files

files.download("test_resultsE.pickle")

***** Running Prediction *****
  Num examples = 357
  Batch size = 16


----------test---------
accuracy =0.597
precision=0.617
recall   =0.592
f1       =0.591


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
print(t_labels_predicted[:10])
print(t_label_ids[:10])

[0 5 8 2 2 8 5 6 8 6]
[0 2 0 1 2 8 5 9 5 6]


In [35]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

In [47]:
a = test_dataset[54]["input_ids"][:50]
print(tokenizer.decode(a))
print(id2label[test_dataset[54]["labels"]])

<s>Diamonds Are Invincible (Mark Ronson x Michael Jackson)
Diamonds Are Invincible (Mark Ronson x Michael Jackson) Lyrics
Keep on with the force don't stop
Don't stop 'til you get enough
Keep
Michael Jackson


In [50]:
print(texts_test[54][:50])
print(labels_test[54])

Diamonds Are Invincible (Mark Ronson x Michael Jac
Michael Jackson
